
# k-NN & Decision Trees Classification

Machine learning classifiers can be broadly categorized into linear and non-linear models. Non-linear classifiers are particularly useful when the relationship between features and target classes is complex or not easily separable by a straight line. Two popular non-linear classifiers are:

-   **k-Nearest Neighbors (k-NN)**
-   **Decision Trees**

## k-Nearest Neighbors (k-NN)

k-Nearest Neighbors is a non-parametric, **instance-based** learning algorithm which does not explicitly learn a model but stores the training instances. For a new observation $\hat{x}$, the algorithm:

1.  Computes the distance between $\hat{x}$ and every point in the training set (commonly the Euclidean distance, but can also use Manhattan or Minkowski distances).
2.  Selects the $k$ closest neighbors.
3.  Predicts the class by majority vote among those neighbors.

The decision boundary is implicitly defined by the $k$ nearest points; therefore the model can capture highly non-linear class boundaries without explicit training.

### Mathematical formulation

Given a training set $\{{(x_i, y_i)}\}_{i = 1}^{n}$ where $y_i \in \{1, \dots,C\}$, the prediction for a query point $\hat{x}$ is $$\hat{y} = \text{mode}\left(\{y_j | x_j \in N_k(\hat{x})\}\right),$$

where $N_k(\hat{x})$ denotes the set of $k$ training points closest to $\hat{x}$.

-   **Choosing $k$**:
    -   Small $k$ can lead to noise sensitivity (overfitting).
    -   Large $k$ can smooth out class boundaries (underfitting).
    -   Cross-validation is often used to select the optimal $k$.
-   **Advantages**:
    -   Simple and intuitive.
    -   No training phase; all computation happens at prediction time.
    -   Naturally handles multi-class classification.
-   **Disadvantages**:
    -   Computationally expensive at prediction time, especially with large datasets.
    -   Sensitive to irrelevant features and feature scaling.
    -   Requires a good choice of distance metric.

## Decision Trees

A Decision Tree recursively partitions the feature space into axis-aligned regions and assigns a class label to each region. It consists of nodes (features), branches (decisions), and leaves (outcomes). At each node the algorithm chooses the split that maximizes a purity metric such as **information gain** or **Gini impurity**. The tree is built by recursively splitting the dataset into subsets until a stopping criterion is met (e.g., maximum depth, minimum samples per leaf).

By hierarchically splitting the space, a tree can approximate complex decision boundaries while remaining interpretable-each internal node corresponds to a human-readable rule.

### Information gain

For a node containing a sample set $S$, the entropy is

$$H(S)= -\sum_{c = 1}^{C} p_c \log_2 p_c,$$

with $p_c$ the proportion of class $c$. A split of $S$ into subsets $S_L, S_R$ yields information gain

$$IG = H(S) - \frac{|S_L|}{|S|} H(S_L) - \frac{|S_R|}{|S|} H(S_R).$$

The algorithm selects the split with the highest **IG** (or lowest Gini).

-   **Advantages**:
    -   Easy to interpret and visualize.
    -   Handles both numerical and categorical data.
    -   Non-parametric: does not assume any specific distribution of the data.
-   **Disadvantages**:
    -   Prone to overfitting, especially with deep trees.
    -   Sensitive to small changes in the data (can lead to different trees).
    -   Can create biased trees if some classes dominate.
-   **Pruning**: Techniques like cost-complexity pruning can be applied to reduce overfitting by removing branches that have little importance.

## Practical Demonstration

Below we illustrate both algorithms on the classic **Iris** data set.

In [ ]:
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd

# Load data and keep only two features for 2-D visualisation
iris = datasets.load_iris()
X = iris.data[:, 2:4]
y = iris.target
feature_names = iris.feature_names[2:4]
class_names = iris.target_names

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

-   Preprocessing the data and training the classifiers

In [ ]:
# Create a Pipeline for k-NN containing standardisation and classifier
knn_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

# Fit k-NN
knn_pipeline.fit(X_train, y_train)

# Predict and evaluate k-NN
y_pred_knn = knn_pipeline.predict(X_test)
print("k-NN accuracy:", accuracy_score(y_test, y_pred_knn))
print(classification_report(y_test, y_pred_knn, target_names=class_names))

In [ ]:
# Create a Pipeline for DecisionTree
dt_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Non-necessary for DT, but included for consistency
    ('dt', DecisionTreeClassifier(max_depth=3, random_state=42))
])

# Fit Decision Tree
dt_pipeline.fit(X_train, y_train)

# Predict and evaluate Decision Tree
y_pred_dt = dt_pipeline.predict(X_test)
print("Decision Tree accuracy:", accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt, target_names=class_names))

-   Visualising the decision boundaries

In [ ]:
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
display = DecisionBoundaryDisplay.from_estimator(
    knn_pipeline, X, response_method='predict',
    cmap=plt.cm.coolwarm, alpha=0.85,
    xlabel=feature_names[0], ylabel=feature_names[1],
    grid_resolution=500)
plt.title("k-NN Decision Boundary")
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolor='k', s=20, cmap=plt.cm.coolwarm)
plt.show()

In [ ]:
display = DecisionBoundaryDisplay.from_estimator(
    dt_pipeline, X, response_method='predict',
    cmap=plt.cm.coolwarm, alpha=0.75,
    xlabel=feature_names[0], ylabel=feature_names[1],
    grid_resolution=500)
plt.title("Decision Tree Decision Boundary")
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolor='k', s=20, cmap=plt.cm.coolwarm)
plt.show()

-   Visualising the Decision Tree structure

In [ ]:
plt.figure(figsize=(12, 8))
plot_tree(dt_pipeline.named_steps['dt'], filled=True, feature_names=feature_names, class_names=class_names)
plt.title("Decision Tree Structure")
plt.show()

## Hands-on Exercises

-   **k-NN hyper-parameters**
    -   Re-run the demo varying $k$ from 1 to 15.
    -   Try Manhattan distance (`metric='manhattan'`) and observe how the boundary changes.
-   **Decision Tree pruning**
    -   Grow an unpruned tree (`max_depth=None`) and visualise it with `plot_tree`.
    -   What, in your opinion, is the disadvantage of an unpruned tree?

-   **Multiclass confusion matrix**
    -   Implement and plot confusion matrices for both classifiers on the full four-feature Iris data set.
        -   Which mis-classifications are most common and why?
-   **Real-world data**
    -   Load the **Wine** data set (`sklearn.datasets.load_wine`).
    -   Explore the data set and visualise the features.
    -   Choose two features for training and visualisation (e.g., alcohol and malic acid).
    -   Build a pipeline with standardisation + k-NN vs. a depth-limited decision tree for the chosen features.
    -   Compare performance using 5-fold cross-validation.